# DB import

### import

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import psycopg2

## 공공데이터포털에서 가져오기

In [2]:

url = 'https://api.odcloud.kr/api/GetGeneralProductInfoService/v1/getGoldPriceInfo?numOfRows=576&pageNo=1&resultType=json&itmsNm=%EA%B8%88%2099.99K&serviceKey=E8xqjRTnATadRGXNxY2AZPEhJZ2iqcbw5%2FYy1g%2BO0jebgbGZDxz5ozNaAeJmc%2F%2Btedi4vF4mBBblOxqJXyYOoQ%3D%3D'
page = requests.get(url)
parsed_data = json.loads(page.text)
item = parsed_data['response']['body']['items']['item']
item
price = [col['clpr'] for col in item]
date = [col['basDt'] for col in item]
gold = pd.DataFrame({'Date' : date,'Price' :price})
gold.sort_values('Date',ascending=True,inplace=True)
gold.reset_index(drop=True,inplace=True)
gold['Date'] = gold['Date'].apply(pd.to_datetime)

## FinanceDataReader에서 데이터 가져오기

In [3]:
import FinanceDataReader as fdr
fdr.__version__

'0.9.34'

In [4]:
def toseries(data):
    data = pd.Series(data['Close'])[:-36]
    return data

In [5]:
kospi = pd.Series(fdr.DataReader('KS11', '2020')['Close'])[:-36]


In [6]:
kosdaq = fdr.DataReader('KQ11','2020')
kosdaq = toseries(kosdaq)

In [7]:
sp500 = fdr.DataReader('US500','2020')
sp500 = toseries(sp500)

In [8]:
nasdaq = fdr.DataReader('IXIC','2020')
nasdaq = toseries(nasdaq)

In [9]:
exchangerate = fdr.DataReader('USD/KRW','2020')
exchangerate = toseries(exchangerate)

In [10]:
indexx = pd.DataFrame({'kospi':kospi,'kosdaq':kosdaq,'s&p500':sp500,'nasdaq':nasdaq,'exchangerate':exchangerate,})
indexx.dropna(inplace=True)
indexx.reset_index(drop=False)

In [13]:
btc = fdr.DataReader('BTC/KRW','2020')
btc = toseries(btc)

btc = pd.DataFrame({'Date':btc.index,'btc':btc.values})

In [14]:
indexi = pd.merge(indexx,btc, how='left',left_on='Date',right_on='Date')

In [15]:
df = pd.merge(indexi,gold, how='left',left_on='Date',right_on='Date')

In [17]:
statistic1 = pd.read_csv('./dataset/date.csv')
statistic2 = pd.read_csv('./dataset/monthly.csv')
statistic3 = pd.read_csv('./dataset/quarter.csv')

In [18]:
bok = statistic1.iloc[0,:][5:-35]

In [19]:
call = statistic1.iloc[1,:][5:-35]

In [20]:
gdp = statistic3.iloc[2,:][5:]

In [21]:
dately = pd.DataFrame({'bok':bok,'call':call})

In [23]:
statistic2 = statistic2.drop(['단위','가중치'], axis=1).dropna()

In [24]:
cpi = statistic2.iloc[1,:][3:]

In [25]:
hdi = statistic2.iloc[2,:][3:]

In [26]:
emi = statistic2.iloc[3,:][3:]

In [27]:
monthly = pd.DataFrame({'cpi':cpi,'phi':hdi,'esi':emi})

## ElephantSQL에 적재

In [79]:
import sqlite3

In [37]:
conn = psycopg2.connect(
    host="jelani.db.elephantsql.com",
    database="zzzdzuqh",
    user="zzzdzuqh",
    password="cg6rqOB5NF2qQ4vqJfQYxxVGh14zIq9k")

In [38]:
cur = conn.cursor()

In [39]:
for i,ii in gold.values:
    cur.execute('INSERT INTO gold(date,price) VALUES (%s,%s)',(i.date(),ii))

In [40]:
conn.commit()

In [41]:
for i,ii in btc.values:
    cur.execute('INSERT INTO btc(date,price) VALUES (%s,%s)',(i.date(),ii))

In [42]:
indexx = indexx.reset_index(drop=False)

In [43]:
for i,ii,iii,ix,v,vi in indexx.values:
    cur.execute('INSERT INTO index_5 (date,kospi,kosdaq,sp500,nasdaq,exchangerate) VALUES (%s,%s,%s,%s,%s,%s)',(i.date(),ii,iii,ix,v,vi))

In [45]:
conn.commit()

In [139]:
monthly.reset_index(drop=False, inplace=True)

In [152]:
monthly['index'] = monthly['index'].apply(pd.to_datetime)

In [153]:
for i,ii,iii,iv in monthly.values:
    cur.execute('INSERT INTO indicator (date, cpi, phi, esi) VALUES (?,?,?,?);',(i.date(),ii,iii,iv))

In [156]:
dately.reset_index(drop=False,inplace=True)

In [162]:
dately['index'] = dately['index'].apply(pd.to_datetime)

In [51]:
dately.drop('call',axis=1,inplace=True)

In [53]:
dately.reset_index(drop=False,inplace=True)

In [60]:
dately['index'] = dately['index'].apply(pd.to_datetime)

In [61]:
for i,ii in dately.values:
    cur.execute('INSERT INTO basemoneyrate (date,bok) VALUES (%s,%s);',(i.date(),ii))

In [62]:
conn.commit()

In [194]:
gdp.values

array([-1.3, -3.0, 2.3, 1.2, 1.7, 0.8, 0.2, 1.3, 0.6], dtype=object)

In [192]:
quarter = pd.DataFrame({'date':monthly['index']})

In [195]:
newdf = pd.DataFrame(np.repeat(gdp.values,3,axis=0))

In [199]:
gdp = pd.concat([quarter,newdf],axis=1)

In [201]:
gdp.columns = ['date','gdp']

In [205]:
gdp = gdp[:-1]

In [206]:
for i,ii in gdp.values:
    cur.execute('INSERT INTO GDP (date,rate) VALUES (?,?);',(i.date(),ii))

In [207]:
conn.commit()